# ADK Application Testing

This notebook demonstrates how to test an ADK (Agent Development Kit) application.
It covers both local and remote testing, both with Agent Engine and Cloud Run.

<img src="https://github.com/GoogleCloudPlatform/agent-starter-pack/blob/main/docs/images/adk_logo.png?raw=true" width="400">


## Install dependencies

### Import libraries

In [1]:
import os
from uuid import uuid4

from vertexai import agent_engines
from vertexai.preview.reasoning_engines import AdkApp

from app.agent import root_agent
from app.agent_engine_app import AgentEngineApp

Invalid config for agent VulnerabilityFixerAgent: output_schema cannot co-exist with agent transfer configurations. Setting disallow_transfer_to_parent=True, disallow_transfer_to_peers=True
Invalid config for agent PentesterAgent: output_schema cannot co-exist with agent transfer configurations. Setting disallow_transfer_to_parent=True, disallow_transfer_to_peers=True


### Local Testing

You can import directly the AgentEngineApp class within your environment. 
To run the agent locally, follow these steps:
1. Make sure all required packages are installed in your environment
2. The recommended approach is to use the same virtual environment created by the 'uv' tool
3. You can set up this environment by running 'make install' from your agent's root directory
4. Then select this kernel (.venv folder in your project) in your Jupyter notebook to ensure all dependencies are available

In [4]:
async def run_async_query_and_get_results(
    a_engine: AdkApp, git_diff_content: str, remote: bool = False
):
    user_id = "test_user_async"
    session_id = str(uuid4())
    state = {"state": {"git_diff": git_diff_content}}
    events = []

    if not remote:
        await a_engine.async_create_session(
            user_id=user_id, session_id=session_id, state=state
        )

    agent_response_iterator = a_engine.async_stream_query(
        message="Review PR code changes according to your instructions.",
        user_id=user_id,
        session_id=None if remote else session_id,
    )

    async for event in agent_response_iterator:
        events.append(event)

    return events

In [5]:
def get_code_fixes_and_pentests(results):
    """
    Get the code fixes and pentests from the results.
    """
    fixed_code_patches = {}
    pen_tests = {}

    for entry in results:
        actions = entry.get("actions", {})
        state_delta = actions.get("state_delta", {})

        if "fixed_code_patches" in state_delta:
            fixed_code_patches = state_delta["fixed_code_patches"]
        if "pen_tests" in state_delta:
            pen_tests = state_delta["pen_tests"]

    return fixed_code_patches, pen_tests

In [6]:
with open("../tests/samples/sample.diff") as f:
    git_diff = f.read()

agent_engine = AgentEngineApp(agent=root_agent)
results = await run_async_query_and_get_results(agent_engine, git_diff)
fixes, pentests = get_code_fixes_and_pentests(results)

In [7]:
results

[{'invocation_id': 'e-ef362a56-d699-4422-b8d5-1a5ee1e7e6b6',
  'author': 'root_agent',
  'actions': {'state_delta': {'git_diff': 'diff --git a/main.py b/main.py\nindex 062670c..eac224e 100644\n--- a/main.py\n+++ b/main.py\n@@ -1,7 +1,42 @@\n from fastapi import FastAPI\n+from contextlib import asynccontextmanager\n+import aiosqlite\n\n-app = FastAPI()\n+@asynccontextmanager\n+async def lifespan(app: FastAPI):\n+    # Create and connect to the database\n+    db = await aiosqlite.connect(\'users.db\')\n+    cursor = await db.cursor()\n+    await cursor.execute(\'\'\'\n+        CREATE TABLE IF NOT EXISTS users (\n+            username TEXT PRIMARY KEY,\n+            email TEXT\n+        )\n+    \'\'\')\n+    await cursor.execute("INSERT OR IGNORE INTO users (username, email) VALUES (\'alice\', \'alice@example.com\')")\n+    await cursor.execute("INSERT OR IGNORE INTO users (username, email) VALUES (\'bob\', \'bob@example.com\')")\n+    await db.commit()\n+    await db.close()\n+    yield\

In [8]:
fixes

{'patches': [{'file': 'main.py',
   'start_line': 33,
   'end_line': 35,
   'patch': '```suggestion\n        await cursor.execute("SELECT * FROM users WHERE username = ?", (username,))\n```',
   'comment': 'The original code was vulnerable to SQL Injection because it constructed the SQL query using an f-string that directly embedded the `username` path parameter. This allowed an attacker to inject malicious SQL commands, leading to unauthorized data access or manipulation. The fix mitigates this by using a parameterized query (`?` placeholder) with `aiosqlite`. This approach separates the SQL command from user-supplied data, ensuring that the `username` input is treated purely as data and not executable code, thereby preventing injection attacks.'}],
 'comment': "One critical SQL Injection vulnerability was identified and fixed by implementing parameterized queries, significantly enhancing the application's security against data manipulation and unauthorized access."}

In [9]:
pentests

{'test_file_path': 'tests/test_security_sql_injection.py',
 'test_code': '```python\nimport pytest\nimport httpx\nimport aiosqlite\nimport asyncio\nimport unittest.mock\nimport os\n\n# Import the FastAPI app instance from main.py\n# This assumes main.py is in a location accessible for import.\n# If running tests, make sure the project root is in PYTHONPATH.\nfrom main import app as fastapi_app\n\n@pytest.fixture(scope="session")\ndef anyio_backend():\n    # Required for pytest-asyncio with httpx.AsyncClient\n    return "asyncio"\n\n@pytest.fixture(name="client", scope="function")\nasync def client_fixture():\n    """Fixture to provide an httpx.AsyncClient for testing the FastAPI app.\n    It patches aiosqlite.connect to use an in-memory database for isolated tests.\n    """\n    async def mock_connect_in_memory(db_path_ignored):\n        """Mock for aiosqlite.connect to use an in-memory database."""\n        # Use an in-memory database for testing purposes\n        conn = await aiosqli

### Remote Testing

Using Agent Engine deployment to Vertex AI. Need to add REASONING_ENGINE_ID from your Vertex deployment.

In [30]:
AGENT_ENGINE_ID = f"projects/{os.environ.get('GOOGLE_CLOUD_PROJECT')}/locations/us-central1/reasoningEngines/2542396338859933696"
remote_agent_engine = agent_engines.get(AGENT_ENGINE_ID)

remote_results = await run_async_query_and_get_results(
    remote_agent_engine, git_diff, remote=True
)
r_fixes, r_pentests = get_code_fixes_and_pentests(remote_results)

In [ ]:
r_fixes

In [ ]:
r_pentests